In [1]:
# fetch_topology.py
import topohub, networkx as nx

# Examples of IDs: 'topozoo/Abilene', 'sndlib/Polska' or 'gabriel/25/0'
# Use topohub.get to fetch a topology dict; TopoHub mirrors Topology Zoo and SNDlib.
TOPO_ID = 'topozoo/Abilene'   # change as desired

topo = topohub.get(TOPO_ID)
G = nx.node_link_graph(topo)   # NetworkX graph with attributes
print("Loaded topology:", topo.get('name', TOPO_ID))
print("Nodes:", len(G.nodes()), "Edges:", len(G.edges()))
# Inspect edge attributes: e.g., G.edges[data=True]
# Save graph for subsequent scripts
import pickle
pickle.dump(G, open("topo_graph.pkl","wb"))


Loaded topology: topozoo/Abilene
Nodes: 11 Edges: 14


In [2]:
# classical_pipeline.py
import pickle, networkx as nx, itertools
from heapq import nsmallest
import pulp, math

G = pickle.load(open("topo_graph.pkl","rb"))

In [3]:
# 1) Normalize / ensure required edge attributes
# If dataset lacks capacity/latency/energy, set defaults:
for u,v,data in G.edges(data=True):
    if 'cap' not in data: data['cap'] = 1000  # Mbps default (tune per topology)
    if 'lat' not in data: data['lat'] = int(data.get('dist',10))  # ms, use 'dist' if available
    if 'energy_per_mbps' not in data: data['energy_per_mbps'] = 0.02  # W per Mbps default


In [4]:
    # 2) Load or synthesize demand matrix
    # If dataset includes 'demands' use it; else synthesize random demands
    demands = topo.get('demands', None) if 'topo' in globals() else None
    if demands is None:
        import random
        random.seed(0)
        demands = []
        nodes = list(G.nodes())
        for i in range(40):  # adjustable
            s,t = random.sample(nodes, 2)
            bw = random.choice([10,20,50,100,200])  # Mbps
            sla = random.choice([30,50,80,120])
            demands.append({'id':f'D{i+1}','src':s,'dst':t,'bw':bw,'sla':sla})


In [5]:
# 3) Candidate paths per demand (k-shortest by latency)
def k_shortest_paths(G, source, target, k=4, weight='lat'):
    # simple Yen's algorithm with networkx if available
    try:
        from networkx.algorithms.simple_paths import shortest_simple_paths
        paths = []
        gen = shortest_simple_paths(G, source, target, weight=weight)
        for _, p in zip(range(k), gen):
            # compute total latency
            lat = sum(G[p[i]][p[i+1]].get('lat',0) for i in range(len(p)-1))
            paths.append({'nodes':p, 'lat':lat})
        return paths
    except Exception:
        # fallback BFS limited depth
        return []

In [6]:
K = 3
P = {}
for d in demands:
    pths = k_shortest_paths(G, d['src'], d['dst'], k=K)
    # Filter SLA-violating paths
    pths = [p for p in pths if p['lat'] <= d['sla']]
    if not pths:
        # if none, take unconstrained shortest path
        try:
            p = nx.shortest_path(G, d['src'], d['dst'], weight='lat')
            lat = sum(G[p[i]][p[i+1]].get('lat',0) for i in range(len(p)-1))
            pths = [{'nodes':p,'lat':lat}]
        except nx.NetworkXNoPath:
            pths=[]
    P[d['id']] = pths

# 4) ILP formulation with drop option (patched)
# ----- PATCH START -----
import pulp

empty = [d['id'] for d in demands if len(P[d['id']]) == 0]
if empty:
    print("Demands with ZERO candidate paths (violating SLA):", empty)
    print("Adding a 'drop' option so the ILP stays feasible.")

x = {}
z = {}
for d in demands:
    for idx, p in enumerate(P[d['id']]):
        x[(d['id'], idx)] = pulp.LpVariable(f"x_{d['id']}_{idx}", cat='Binary')
    z[d['id']] = pulp.LpVariable(f"drop_{d['id']}", cat='Binary')

def path_energy(G, nodes, bw):
    return sum(G[nodes[i]][nodes[i+1]]['energy_per_mbps'] * bw for i in range(len(nodes)-1))

DROP_PENALTY = 1e6
objective_terms = []
for d in demands:
    for idx in range(len(P[d['id']])):
        objective_terms.append(path_energy(G, P[d['id']][idx]['nodes'], d['bw']) * x[(d['id'], idx)])
    objective_terms.append(DROP_PENALTY * z[d['id']])

prob = pulp.LpProblem("EnergyAwareRouting", pulp.LpMinimize)
prob += pulp.lpSum(objective_terms)

for d in demands:
    prob += pulp.lpSum([x[(d['id'], idx)] for idx in range(len(P[d['id']]))]) + z[d['id']] == 1

for u, v, data in G.edges(data=True):
    prob += pulp.lpSum([
        d['bw'] * x[(d['id'], idx)]
        for d in demands for idx in range(len(P[d['id']]))
        if any((a == u and b == v) or (a == v and b == u)
               for a, b in zip(P[d['id']][idx]['nodes'][:-1], P[d['id']][idx]['nodes'][1:]))
    ]) <= data['cap']

cbc_path = "/opt/homebrew/opt/cbc/bin/cbc" 
prob.solve(pulp.COIN_CMD(path=cbc_path, msg=True, timeLimit=60))
print("Solver status:", pulp.LpStatus[prob.status])

sol = {}
total_energy = 0.0
dropped = []
for d in demands:
    if pulp.value(z[d['id']]) and pulp.value(z[d['id']]) > 0.5:
        dropped.append(d['id'])
        continue
    for idx in range(len(P[d['id']])):
        if pulp.value(x[(d['id'], idx)]) > 0.5:
            sol[d['id']] = idx
            total_energy += path_energy(G, P[d['id']][idx]['nodes'], d['bw'])
            break

def link_utilization(G, sol):
    util = {(u, v): 0.0 for u, v in G.edges()}
    for d in demands:
        idx = sol.get(d['id'])
        if idx is None:
            continue
        nodes = P[d['id']][idx]['nodes']
        for a, b in zip(nodes[:-1], nodes[1:]):
            e = (a, b) if (a, b) in util else (b, a)
            util[e] += d['bw']
    util_pct = {e: 100.0 * load / G[e[0]][e[1]]['cap'] for e, load in util.items()}
    return util, util_pct

util_abs, util_pct = link_utilization(G, sol)
print(f"ILP result (with drop option): total energy = {total_energy:.3f}")
print(f"Dropped demands: {len(dropped)}")
print(f"Max link utilization: {max(util_pct.values()):.1f}%")
# ----- PATCH END -----


Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - /opt/homebrew/opt/cbc/bin/cbc /var/folders/c8/gcpq5wfj6vqf9rqk1jhxvqq80000gn/T/dc5fc9fbe2514ddc95eabf5ecbef3af1-pulp.mps -sec 60 -timeMode elapsed -branch -printingOptions all -solution /var/folders/c8/gcpq5wfj6vqf9rqk1jhxvqq80000gn/T/dc5fc9fbe2514ddc95eabf5ecbef3af1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 59 COLUMNS
At line 491 RHS
At line 546 BOUNDS
At line 627 ENDATA
Problem MODEL has 54 rows, 80 columns and 191 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.55012e+06 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 4 rows, 24 columns (24 integer (21 of which binary)) and 53 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.45
Cbc0038I Solution found of 3

In [7]:
import pickle

# After computing 'demands' and 'P' in classical code
with open("demands.pkl", "wb") as f:
    pickle.dump(demands, f)

with open("candidate_paths.pkl", "wb") as f:
    pickle.dump(P, f)

print("Saved demands and candidate paths for QAOA pipeline.")
import pickle

demands = pickle.load(open("demands.pkl", "rb"))
P = pickle.load(open("candidate_paths.pkl", "rb"))

# Check for ellipsis in demands
for d in demands:
    for k, v in d.items():
        if v is ...:
            print("Found ellipsis in demands:", d)

# Check for ellipsis in P
for k, paths in P.items():
    for p in paths:
        for kk, vv in p.items():
            if vv is ...:
                print("Found ellipsis in P:", k, p)

Saved demands and candidate paths for QAOA pipeline.


In [8]:
# qaoa_pipeline_small.py
import pickle
import numpy as np
from numbers import Number

# Qiskit imports
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliSumOp

# -------------------------------
# Load topology graph and classical outputs
# -------------------------------
G = pickle.load(open("topo_graph.pkl", "rb"))

# Fill missing edge attributes
for u, v, data in G.edges(data=True):
    if 'cap' not in data:
        data['cap'] = 1000  # Mbps
    if 'lat' not in data:
        data['lat'] = int(data.get('dist', 10))  # ms
    if 'energy_per_mbps' not in data:
        data['energy_per_mbps'] = 0.02  # W per Mbps

# Load classical outputs
demands_full = pickle.load(open("demands.pkl", "rb"))
P_full = pickle.load(open("candidate_paths.pkl", "rb"))

# -------------------------------
# Reduce problem size (limit qubits)
# -------------------------------
MAX_DEMANDS = 4     # pick first 4 demands
MAX_PATHS = 3       # pick first 3 paths per demand

demands = demands_full[:MAX_DEMANDS]
P = {}
for d in demands:
    P[d['id']] = P_full[d['id']][:MAX_PATHS]

# -------------------------------
# Build variable index mapping
# -------------------------------
var_index = {}
rev_var = {}
idx = 0
for d in demands:
    for p_idx in range(len(P[d['id']])):
        var_index[(d['id'], p_idx)] = idx
        rev_var[idx] = (d['id'], p_idx)
        idx += 1
nvars = idx
print(f"Reduced problem size: {nvars} variables (~{nvars} qubits)")

# -------------------------------
# Compute energy, edge incidence, latency
# -------------------------------
var_energy = np.zeros(nvars)
var_edge_incidence = [dict() for _ in range(nvars)]
var_latency = np.zeros(nvars)

for (did, pidx), v in var_index.items():
    d = next(x for x in demands if x['id'] == did)
    nodes = P[did][pidx]['nodes']
    lat = P[did][pidx]['lat']
    var_latency[v] = lat
    e_energy = 0.0
    inc = {}
    for i in range(len(nodes)-1):
        u, vv = nodes[i], nodes[i+1]
        ekey = (u, vv) if (u, vv) in G.edges else (vv, u)
        bw = d['bw']
        inc[ekey] = inc.get(ekey, 0) + bw
        e_energy += G[u][vv]['energy_per_mbps'] * bw
    var_energy[v] = e_energy
    var_edge_incidence[v] = inc

# -------------------------------
# Build QUBO dictionary
# -------------------------------
Q = {}
const = 0.0

# Linear energy terms
for i in range(nvars):
    Q[(i, i)] = Q.get((i, i), 0.0) + float(var_energy[i])

# One-hot constraints
penalty_A = 1e5
for d in demands:
    vars_d = [var_index[(d['id'], p)] for p in range(len(P[d['id']]))]
    for i in vars_d:
        Q[(i, i)] = Q.get((i, i), 0.0) + penalty_A
        for j in vars_d:
            if j <= i:
                continue
            Q[(i, j)] = Q.get((i, j), 0.0) + 2 * penalty_A
    const += penalty_A * 1

# Capacity penalties
penalty_B = 500.0
for u, v, edata in G.edges(data=True):
    cap = edata['cap']
    vars_aff = [i for i in range(nvars) if (u, v) in var_edge_incidence[i] or (v, u) in var_edge_incidence[i]]
    for i in vars_aff:
        bw_i = var_edge_incidence[i].get((u, v), 0) + var_edge_incidence[i].get((v, u), 0)
        Q[(i, i)] = Q.get((i, i), 0.0) + penalty_B*(bw_i**2) - 2*penalty_B*cap*bw_i
        for j in vars_aff:
            if j <= i:
                continue
            bw_j = var_edge_incidence[j].get((u, v), 0) + var_edge_incidence[j].get((v, u), 0)
            Q[(i, j)] = Q.get((i, j), 0.0) + 2*penalty_B*bw_i*bw_j
    const += penalty_B*(cap**2)

# SLA penalties
penalty_lat = 1e6
for i in range(nvars):
    did, pidx = rev_var[i]
    d = next(x for x in demands if x['id'] == did)
    if var_latency[i] > d['sla']:
        Q[(i, i)] = Q.get((i, i), 0.0) + penalty_lat

# -------------------------------
# Convert QUBO to Ising coefficients
# -------------------------------
def qubo_to_ising(Q, nvars):
    a = np.zeros(nvars)
    b = {}
    const = 0.0
    for (i, j), coeff in Q.items():
        coeff = float(coeff)
        if i == j:
            const += 0.5*coeff
            a[i] += -0.5*coeff
        else:
            const += 0.25*coeff
            a[i] += -0.25*coeff
            a[j] += -0.25*coeff
            b[(i, j)] = b.get((i, j), 0.0) + 0.25*coeff
    return a, b, const

a, b, const = qubo_to_ising(Q, nvars)

# -------------------------------
# Build PauliSumOp safely
# -------------------------------
pauli_list = []

for i in range(nvars):
    coeff = float(a[i])
    if abs(coeff) < 1e-12:
        continue
    label = ['I'] * nvars
    label[nvars-1-i] = 'Z'
    pauli_list.append((''.join(label), coeff))

for (i, j), c in b.items():
    coeff = float(c)
    if abs(coeff) < 1e-12:
        continue
    label = ['I'] * nvars
    label[nvars-1-i] = 'Z'
    label[nvars-1-j] = 'Z'
    pauli_list.append((''.join(label), coeff))

# Safety check
if len(pauli_list) == 0:
    pauli_list.append(('I'*nvars, 0.0))

H = PauliSumOp.from_list(pauli_list)

# -------------------------------
# Run QAOA
# -------------------------------
qi = QuantumInstance(Aer.get_backend('aer_simulator'), shots=1024)
qaoa = QAOA(optimizer=COBYLA(maxiter=100), reps=2, quantum_instance=qi)

res = qaoa.compute_minimum_eigenvalue(operator=H)

print("QAOA result:", res)
print("QUBO constant term:", const)

Reduced problem size: 4 variables (~4 qubits)
QAOA result: {   'aux_operator_eigenvalues': None,
    'cost_function_evals': 45,
    'eigenstate': {   '0000': 0.16828640022295324,
                      '0001': 0.06987712429686843,
                      '0100': 0.0625,
                      '0101': 0.3451675317871019,
                      '0111': 0.05412658773652741,
                      '1000': 0.3590351654086268,
                      '1001': 0.1795175827043134,
                      '1010': 0.0625,
                      '1011': 0.05412658773652741,
                      '1100': 0.15625,
                      '1101': 0.793036096278095,
                      '1110': 0.03125,
                      '1111': 0.13621559198564606},
    'eigenvalue': (-151011715.37578133+0j),
    'optimal_circuit': None,
    'optimal_parameters': {   ParameterVectorElement(β[0]): 0.026965724448055596,
                              ParameterVectorElement(γ[0]): -1.695455351518141,
                            